<a href="https://colab.research.google.com/github/almeida-cma/TI_TEMA01/blob/main/TI_T1_Cap_Faces.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title 📸 Captura de rostos com contagem
from IPython.display import display, Javascript, Image, clear_output
from google.colab.output import eval_js
from base64 import b64decode
import cv2
import numpy as np
from PIL import Image as PILImage
import io

# Captura imagem da câmera e retorna como arquivo
def take_photo():
  js = Javascript('''
    async function takePhoto() {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = "📸 Capturar";
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      div.appendChild(video);

      document.body.appendChild(div);

      const stream = await navigator.mediaDevices.getUserMedia({ video: true });
      video.srcObject = stream;
      await video.play();

      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);

      stream.getTracks().forEach(track => track.stop());
      div.remove();

      return canvas.toDataURL('image/jpeg');
    }

    takePhoto();
  ''')

  display(js)
  data = eval_js("takePhoto()")  # Aqui está o segredo: avalia a função da string
  binary = b64decode(data.split(',')[1])
  return binary

# Captura e decodifica imagem
image_bytes = take_photo()
image = PILImage.open(io.BytesIO(image_bytes))
image_np = np.array(image)
gray = cv2.cvtColor(image_np, cv2.COLOR_RGB2GRAY)

# Detecta rostos
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
faces = face_cascade.detectMultiScale(gray, 1.3, 5)

# Desenha caixas
for (x, y, w, h) in faces:
  cv2.rectangle(image_np, (x, y), (x+w, y+h), (0, 255, 0), 2)

# Exibe resultado
clear_output()
print(f"✅ Rostos detectados: {len(faces)}")
display(PILImage.fromarray(image_np))
